In [1]:
#import ipywidgets
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, IntSlider
from IPython.display import clear_output

In [2]:
url = 'https://drive.google.com/file/d/1vuIGiBWieIcheFwg2HYMPN2R5I3KWEpX/view?usp=sharing'
path_tidy = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path_tidy)

In [3]:
df.head()

,event_Idx,period,periodTime,id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,rinkSide_of_the_team_that_shot,goalie_name,...,empty_net,strength,type_season,id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name
0,5,1,00:16,8,Montréal Canadiens,Shot,-8.0,-36.0,right,Henrik Lundqvist,...,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens
1,11,1,01:56,3,New York Rangers,Shot,85.0,-6.0,left,Carey Price,...,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens
2,12,1,02:04,8,Montréal Canadiens,Shot,-69.0,-35.0,right,Henrik Lundqvist,...,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens
3,21,1,03:40,8,Montréal Canadiens,Shot,-55.0,-17.0,right,Henrik Lundqvist,...,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens
4,32,1,04:56,8,Montréal Canadiens,Shot,-58.0,-28.0,right,Henrik Lundqvist,...,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens


In [4]:
#SET DEFAULT VALUES
season =  20162017
seasonW = Dropdown(options = df['season'].unique(), value = season)

game_type = 'regular_season'
game_typeW = Dropdown(options=['playoffs','regular_season'], value = game_type)
 
games = df[(df['season']==season) & (df['type_season']==game_type)]['id_game'].unique()
gameW = IntSlider(value = 0,min=0,max=len(games)-1,continuous_update=False)

events = df[(df['season']==season) & (df['type_season']==game_type) & (df['id_game']==games[0])]
eventW = IntSlider(value = 0,min=0,max=len(events)-1,continuous_update=False)

#INTERACTIF DEBUGGER
@interact(season = seasonW,game_type = game_typeW,game = gameW,event=eventW)
def debugger(season,game_type,game,event):
    
    #RESET GAMES
    games = df[(df['season']==season) & (df['type_season']==game_type)]['id_game'].unique()
    gameW.max = len(games) - 1
    
    if game > len(games):
        game = 0
        gameW.value = 0
        
    #RESET EVENTS
    events = df[(df['season']==season) & (df['type_season']==game_type) & (df['id_game']==games[game])]
    eventW.max =  len(events) - 1
    
    if event >  len(events):
        event = 0
        eventW.value = 0
        
    #GET INFO FOR THIS PARTICULAR EVENT
    event_infos = events.iloc[event]
    
    #PLOT BACKGROUND IMAGE
    fig, ax = plt.subplots()
    img = plt.imread("../figures/nhl_rink.png")
    ax.imshow(img, extent=[-100, 100, -43, 43])
    
    #SUPTITLE
    season_title = f'{str(season)[:4]}-{str(season)[4:]}'
    home = event_infos['team_home_name']
    away = event_infos['team_away_name']
   
    plt.suptitle(f'{season_title}\n\n{home} VS {away}')
    
    #TITLE
    shot_type =  event_infos['shot_type']
    event_type =  event_infos['result_event']
    player =  event_infos['shooter_name']
     
    plt.title(f'A {shot_type} of {player}: {event_type}')
   
   
    #PLOT POINT AT THE LOCATION OF THE EVENT
    x =  event_infos['x_coord']
    y =  event_infos['y_coord']
    ax.scatter(x, y, marker='o', color='r', label='point')
    
    #REMOVE PAST PLOTS IF ANY
    clear_output(wait=True)

interactive(children=(Dropdown(description='season', options=(20162017, 20172018, 20182019, 20192020, 20202021…